In [1]:
import barmuscomp.ae_utils as ae_utils
import barmuscomp.ae as ae
import barmuscomp.lra as lra
import barmuscomp.scripts.default_path as paths
import barmuscomp.scripts.overall_scripts as scr

import as_seg.CBM_algorithm as CBM
import as_seg.barwise_input as bi
import as_seg.data_manipulation as dm
import as_seg.autosimilarity_computation as as_comp
#import as_seg.barwise_input as bi
from barmuscomp.model.current_plot import *

import math
import numpy as np
import pandas as pd
import mirdata
import os
import tensorly as tl

In [2]:
colors = ['tab:green', 'tab:purple','tab:red','tab:olive']
markers = ['^', 's', 'o','v']

bands_number = 7
autosimilarity_types = ["Cosine", "Covariance", "RBF"]
pen_function = "modulo8"
feature = "nn_log_mel_grill"
init_nmf = "nndsvd"

compression_labels = ["NMF"]#, "PCA", "SSAE"]

dimensions_range = [8,16,24,32,40]

In [3]:
def return_results(references_segments, segments_in_time):
    prec05, rap05, f_mes05 = dm.compute_score_of_segmentation(references_segments, segments_in_time, window_length = 0.5)
    prec3, rap3, f_mes3 = dm.compute_score_of_segmentation(references_segments, segments_in_time, window_length = 3)
    return [[round(prec05,4),round(rap05,4),round(f_mes05,4)], [round(prec3,4),round(rap3,4),round(f_mes3,4)]]

In [4]:
def plot_kernels_as(results_diff_as, kernels_or_features, autosimilarities, tol, title = None):
    # Order results_diff_as: kernels ou features, as, tol
    subaxis = autosimilarities
    plt.rcParams.update({'font.size': 14})
    
    width=0.25
    gap = (len(autosimilarities) + 1)*width
    colors = ["#62d5b4", "#317373", "#184a4a"]#['skyblue', 'steelblue', 'darkorange', 'brown']
    
    fig, ax = plt.subplots(1, 1, figsize=(11,4))
    ax.set_ylim( 25, 105 )
    tab = []
    current_start = 0
    for i in range(len(autosimilarities)):
        indexes = [current_start + j*gap for j in range(len(kernels_or_features))]
        tab.append(ax.bar(indexes, results_diff_as[:,i,tol], color=colors[i], width=width, align='edge'))
        for idx_x, x in enumerate(indexes):
            height = results_diff_as[idx_x, i, tol] 
            plt.text(x +width/2,height + 2, f"{height}%", ha='center', va='bottom', rotation = 90)
        current_start += width

    ax.set_xticks([current_start/2 + j*gap for j in range(len(kernels_or_features))])
    ax.set_xticklabels(kernels_or_features)
    if tol == 1:
        ax.legend(labels=subaxis, loc='lower left')
    else:
        ax.legend(labels=subaxis, loc='lower left')

    ax.set_ylabel("F measure (percentage)")
    #fig.savefig("C:/Users/amarmore/Desktop/Mes papiers/figures/results_barcomp_05.pdf", bbox_inches = "tight")#, format='png')
    if title != None:
        plt.title(title)
    plt.show()

# EUC

In [5]:
def compute_this_penalty_compression_euc(penalty_func, dim_patterns, autosimilarity_type = "cosine", feature = "nn_log_mel_grill", bands_number = 7):
    idx_song_max = 101 # Should be 101 for all the dataset, other values should be for tests only
    songs_range = range(1,idx_song_max) 
    subdivision = 96
    hop_length = 32
    hop_length_seconds = hop_length/44100
    lambda_penalty = 1

    #results_songs_methods = math.inf * np.ones((len(songs_range), 1, len(dim_patterns), 2, 3)) # Songs, autosimilarity types, tol, metrics
    results_songs_nmf = math.inf * np.ones((len(songs_range), len(dim_patterns), 2, 3)) # Songs, autosimilarity types, tol, metrics

    for idx_song, song_name in enumerate(songs_range):
        spectrogram, bars, references_segments = scr.load_spec_annot_song_RWC(song_name, feature, hop_length)
        
        for idx_dc, dc in enumerate(dim_patterns):
            ### NMF
            #idx_method = 0
            Q_matrix_nmf = np.load(f"{paths.path_data_persisted_rwc}/nmf/Q_nmf_dim{dc}_updatehals_init{init_nmf}_song{song_name}_feature{feature}_hop{hop_length}_subdivbars{subdivision}.npy", allow_pickle = True)

            autosimilarity_nmf = as_comp.switch_autosimilarity(Q_matrix_nmf, similarity_type = autosimilarity_type, normalise = True)
            segments_nmf = CBM.compute_cbm(autosimilarity_nmf, penalty_weight = lambda_penalty, penalty_func = penalty_func, bands_number = bands_number)[0]                
            segments_nmf_in_time = dm.segments_from_bar_to_time(segments_nmf, bars)
            results_songs_nmf[idx_song, idx_dc] = return_results(references_segments, segments_nmf_in_time)
            
    odd_songs = range(0,idx_song_max - 1,2) # Weird, but shouldn't confuse indexes and number
    even_songs = range(1,idx_song_max - 1,2)
    to_return = -math.inf * np.ones((2, 3))
    
    #for idx_method in range(1):
    avg_odd_songs = np.mean(results_songs_nmf[odd_songs, :,0, 2], axis = 0) + np.mean(results_songs_nmf[odd_songs, :,1, 2], axis = 0)

    best_idx_odd = np.argmax(avg_odd_songs)
    print(f"Odd, method {compression_labels[0]}, best dc: {dim_patterns[best_idx_odd]}")
    res_odd = math.inf * np.ones((2,3))
    for tol in range(2):
        for metric in range(3):
            res_odd[tol, metric] = np.mean(results_songs_nmf[even_songs, best_idx_odd, tol, metric])

    avg_even_songs = np.mean(results_songs_nmf[even_songs, :,0, 2], axis = 0) + np.mean(results_songs_nmf[even_songs,:,1, 2], axis = 0)
    best_idx_even = np.argmax(avg_even_songs)
    print(f"Even, method {compression_labels[0]}, best dc: {dim_patterns[best_idx_even]}")
    res_even = math.inf * np.ones((2,3))
    for tol in range(2):
        for metric in range(3):
            res_even[tol, metric] = np.mean(results_songs_nmf[odd_songs, best_idx_even, tol, metric])

    to_return = np.mean(np.array([res_odd, res_even]), axis=0)
    return to_return

In [6]:
feature = "nn_log_mel_grill"

print(f"Kernel: {bands_number}")

res_avg_target_euc = math.inf * np.ones((len(compression_labels), len(autosimilarity_types), 2, 3))

indexes = []
results_this_function = math.inf * np.ones((len(autosimilarity_types), 2, 3))
for idx_as, autosimil_type in enumerate(autosimilarity_types):
    print(f"Autosimilarity type: {autosimil_type}")
    res = compute_this_penalty_compression_euc(pen_function, dimensions_range, autosimilarity_type = autosimil_type, feature = feature, bands_number = bands_number)
    #for idx_line, line in enumerate(res):
        #res_avg_target_euc[idx_line, idx_as] = line
    dataframe = pd.DataFrame(res.reshape(1, 6), columns = ['P0.5', 'R0.5', 'F0.5','P3', 'R3', 'F3'], index = compression_labels)
    display(dataframe.style.bar(subset=["F0.5","F3"], color='#5fba7d'))
#plot_kernels_as(np.round(100 * res_avg_target_euc[:,:,:,2], decimals = 2), compression_labels, autosimilarity_types, 0, title = f"F0.5 for the Euclidean-based compression methods,\nand the different autosimilarities on RWC Pop.")
#plot_kernels_as(np.round(100 * res_avg_target_euc[:,:,:,2], decimals = 2), compression_labels, autosimilarity_types, 1, title = f"F3 for the Euclidean-based compression methods,\nand the different autosimilarities on RWC Pop.")
#print("---------------\n------------\n------------\n------------\n------------")

Kernel: 7
Autosimilarity type: Cosine
Odd, method NMF, best dc: 8
Even, method NMF, best dc: 8


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5636,0.6187,0.5838,0.7069,0.7785,0.7334


Autosimilarity type: Covariance
Odd, method NMF, best dc: 8
Even, method NMF, best dc: 16


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.3923,0.7024,0.4948,0.4842,0.8700,0.6113


Autosimilarity type: RBF
Odd, method NMF, best dc: 8
Even, method NMF, best dc: 8


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5233,0.6646,0.5769,0.6495,0.8281,0.7173


# BETA

In [7]:
def compute_this_penalty_compression_beta(penalty_func, dim_patterns, autosimilarity_type = "cosine", feature = "nn_log_mel_grill", bands_number = 7, beta = 1):
    idx_song_max = 101 # Should be 101 for all the dataset, other values should be for tests only
    songs_range = range(1,idx_song_max) 
    subdivision = 96
    hop_length = 32
    hop_length_seconds = hop_length/44100
    lambda_penalty = 1

    results_songs_methods = math.inf * np.ones((len(songs_range), len(dim_patterns), 2, 3)) # Songs, autosimilarity types, tol, metrics

    for idx_song, song_name in enumerate(songs_range):
        spectrogram, bars, references_segments = scr.load_spec_annot_song_RWC(song_name, feature, hop_length)
        barwise_TF_matrix = bi.barwise_TF_matrix(spectrogram, bars, hop_length_seconds, subdivision)
        
        for idx_dc, dc in enumerate(dim_patterns):
            ### NMF
            #idx_method = 0
            Q_matrix_nmf = np.load(f"{paths.path_data_persisted_rwc}/nmf/Q_nmf_dim{dc}_updatemu_beta{beta}_init{init_nmf}_song{song_name}_feature{feature}_hop{hop_length}_subdivbars{subdivision}.npy", allow_pickle = True)

            autosimilarity_nmf = as_comp.switch_autosimilarity(Q_matrix_nmf, similarity_type = autosimilarity_type, normalise = True)
            segments_nmf = CBM.compute_cbm(autosimilarity_nmf, penalty_weight = lambda_penalty, penalty_func = penalty_func, bands_number = bands_number)[0]                
            segments_nmf_in_time = dm.segments_from_bar_to_time(segments_nmf, bars)
            results_songs_methods[idx_song, idx_dc] = return_results(references_segments, segments_nmf_in_time)
                        
    odd_songs = range(0,idx_song_max - 1,2) # Weird, but shouldn't confuse indexes and number
    even_songs = range(1,idx_song_max - 1,2)
    to_return = -math.inf * np.ones((2, 2, 3))
    
    #for idx_method in range(1):
    avg_odd_songs = np.mean(results_songs_methods[odd_songs,:,0, 2], axis = 0) + np.mean(results_songs_methods[odd_songs,:,1, 2], axis = 0)

    best_idx_odd = np.argmax(avg_odd_songs)
    print(f"Odd, method {compression_labels[0]}, best dc: {dim_patterns[best_idx_odd]}")
    res_odd = math.inf * np.ones((2,3))
    for tol in range(2):
        for metric in range(3):
            res_odd[tol, metric] = np.mean(results_songs_methods[even_songs, best_idx_odd, tol, metric])

    avg_even_songs = np.mean(results_songs_methods[even_songs,:,0, 2], axis = 0) + np.mean(results_songs_methods[even_songs,:,1, 2], axis = 0)
    best_idx_even = np.argmax(avg_even_songs)
    print(f"Even, method {compression_labels[0]}, best dc: {dim_patterns[best_idx_even]}")
    res_even = math.inf * np.ones((2,3))
    for tol in range(2):
        for metric in range(3):
            res_even[tol, metric] = np.mean(results_songs_methods[odd_songs, best_idx_even, tol, metric])

    to_return = np.mean(np.array([res_odd, res_even]), axis=0)
    
    return to_return

## Beta 1

In [8]:
feature = "nn_log_mel_grill"

print(f"Kernel: {bands_number}")

beta_compression_labels = ["NMF"]#, "SSAE"]
res_avg_target_kl = math.inf * np.ones((len(beta_compression_labels), len(autosimilarity_types), 2, 3))

indexes = []
results_this_function = math.inf * np.ones((len(autosimilarity_types), 2, 3))
for idx_as, autosimil_type in enumerate(autosimilarity_types):
    print(f"Autosimilarity type: {autosimil_type}")
    res = compute_this_penalty_compression_beta(pen_function, dimensions_range, autosimilarity_type = autosimil_type, feature = feature, bands_number = bands_number, beta = 1)
    #for idx_line, line in enumerate(res):
        #res_avg_target_kl[idx_line, idx_as] = line
    dataframe = pd.DataFrame(res.reshape(1, 6), columns = ['P0.5', 'R0.5', 'F0.5','P3', 'R3', 'F3'], index = beta_compression_labels)
    display(dataframe.style.bar(subset=["F0.5","F3"], color='#5fba7d'))

#plot_kernels_as(np.round(100 * res_avg_target_kl[:,:,:,2], decimals = 2), beta_compression_labels, autosimilarity_types, 0, title = f"F0.5 for KL-based compression methods,\nand the different autosimilarities on RWC Pop.")
#plot_kernels_as(np.round(100 * res_avg_target_kl[:,:,:,2], decimals = 2), beta_compression_labels, autosimilarity_types, 1, title = f"F3 for KL-based compression methods,\nand the different autosimilarities on RWC Pop.")
#print("---------------\n------------\n------------\n------------\n------------")

Kernel: 14_bands
Autosimilarity type: Cosine
Odd, method NMF, best dc: 40
Even, method NMF, best dc: 40


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.6121,0.6299,0.6164,0.7902,0.8161,0.7975


Autosimilarity type: Covariance
Odd, method NMF, best dc: 40
Even, method NMF, best dc: 24


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5401,0.6426,0.5789,0.6906,0.8235,0.7414


Autosimilarity type: RBF
Odd, method NMF, best dc: 8
Even, method NMF, best dc: 16


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5468,0.5852,0.5604,0.7227,0.7774,0.7429


## Beta 0

In [9]:
feature = "nn_log_mel_grill"

print(f"Kernel: {bands_number}")

beta_compression_labels = ["NMF"]
res_avg_target_is = math.inf * np.ones((len(beta_compression_labels), len(autosimilarity_types), 2, 3))

indexes = []
results_this_function = math.inf * np.ones((len(autosimilarity_types), 2, 3))
for idx_as, autosimil_type in enumerate(autosimilarity_types):
    print(f"Autosimilarity type: {autosimil_type}")
    res = compute_this_penalty_compression_beta(pen_function, dimensions_range, autosimilarity_type = autosimil_type, feature = feature, bands_number = bands_number, beta = 0)
    #for idx_line, line in enumerate(res):
        #res_avg_target_is[idx_line, idx_as] = line
    dataframe = pd.DataFrame(res.reshape(1, 6), columns = ['P0.5', 'R0.5', 'F0.5','P3', 'R3', 'F3'], index = beta_compression_labels)
    display(dataframe.style.bar(subset=["F0.5","F3"], color='#5fba7d'))
#plot_kernels_as(np.round(100 * res_avg_target_is[:,:,:,2], decimals = 2), beta_compression_labels, autosimilarity_types, 0, title = f"F0.5 for IS-based compression methods,\nand the different autosimilarities on RWC Pop.")
#plot_kernels_as(np.round(100 * res_avg_target_is[:,:,:,2], decimals = 2), beta_compression_labels, autosimilarity_types, 1, title = f"F3 for IS-based compression methods,\nand the different autosimilarities on RWC Pop.")
#print("---------------\n------------\n------------\n------------\n------------")

Kernel: 14_bands
Autosimilarity type: Cosine
Odd, method NMF, best dc: 24
Even, method NMF, best dc: 16


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5744,0.5993,0.5823,0.7621,0.7952,0.7730


Autosimilarity type: Covariance
Odd, method NMF, best dc: 24
Even, method NMF, best dc: 16


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5264,0.6157,0.5611,0.6846,0.8037,0.7314


Autosimilarity type: RBF
Odd, method NMF, best dc: 8
Even, method NMF, best dc: 16


,P0.5,R0.5,F0.5,P3,R3,F3
NMF,0.5415,0.5883,0.5590,0.7224,0.7855,0.7467
